# Quantum computing basics and Benchmarking Random Circuits

In this tutorial we will explain some of the problems encountered when trying to define benchmarks for quantum software and we will have a look at some performance metrics of two well quantum python libraries: Qiskit (Abraham et al., 2019(https://qiskit.org/)) and Qulacs (“Qulacs”, 2018(https://github.com/qulacs/qulacs)). 


## Contents 

1. [Introduction](#introduction)

    1.1. [Quantum computing](#quantumcomp)
    
    1.2. [Benchmarking quantum simulators](#quantumsim)
    
    1.3. [Random Circuits](#randomcircuits)
    
    1.4. [Qiskit and Qulacs](#qulacs)
    
  
2. [Benchmark examples](#examples)

    2.1. [Hadamards](#hadamards)
    
    2.2. [CNOT](#cnot)
    
    2.3. [Timing gates](#timing)
    

3. [Random circuits](#randomcircuit)
        
    3.1. [Random circuit 1 time](#RC1time)
    
    3.2. [Random circuit 1 memory](#RC1mem)
    
    3.3. [Random circuit 2 time](#RCcliftime)
    
    3.4. [Random circuit 2 memory](#RCclifmem)
    
    3.5. [Running on a QPU](#ibmQPU)
    
3. [Conclusion](#conclusion)

## 1. Introduction <a id='introduction'></a>

### 1.1. Quantum computing <a id='quantumcomp'></a>
In recent years quantum computing has become a buzzword and the idea of a fully working quantum computer is very exciting.  Quantum knowledge and technology has grown exponentially fast but still
developing a non-trivial size quantum computer is not straight forward. We will probably have to wait some more years to witness their power and applicability. 

The quantum computers available nowadays (eg. IBM, Google, QuTech, etc) are what we call Noisy Intermediate-Scale Quantum (NISQ) devices. They are quantum computers capable of executing quantum algorithms and calculations but where the sources of errors (eg. due to gates noise, coherence, calibration errors, etc.) limit how reliable they are. NISQ devices can be benchmarked for example using the "quantum volume". The quantum volume takes into account not only the number of qubits (width of a circuit) but also the number of operations (depth of a circuit) a quantum computer can handle. The depth of a circuit, is influenced by things such as the number of qubits, how qubits are interconnected, gate and measurement errors, device cross talk, circuit compiler efficiency, and more.
Therefore, the quantum volume indicates the relative complexity of problems that can be solved by the quantum computer. The larger the number, the more powerful the quantum computer is.

Information in a quantum computer is stored in qubits.
A qubit is a two state system with quantum properties. As opposed to a bit that can only be in state 0 or 1 (on or off), a state of a qubit (denoted $|\psi>$) can be in a combination (superposition) of the two states (called basis states),

$|\psi> = \alpha|0> + \beta|1>$

The qubit is therefore not 0 or 1 (on or off), but a little bit of both (like a dimmy light). How on or how off it is depends on the coefficients $\alpha$ and $\beta$.

When a qubit is measured we "force" it to become (collpase it to) one of the basis states of the measurment. 
The probability of us measuring one state or the other (|0> or |1>) depends on how they contribute to the total state in the first place: depends on the values of $\alpha$ and $\beta$. The higher the value the more likely it is we measure the associated state. These numbers are known as probability amplitudes and the probability of measuring a given state is given by $\alpha^2$ and $\beta^2$. Since the total probability of measuring one state is 1.0, $\alpha^2 +\beta^2 = 1$. 

As opposed to a classical bit, whcih can be described by only one number: 0 or 1, a qubit needs two numbers to be fully described: the probability amplitudes. Therefore, a qubit can be represented as a vector with two entries:

$|\psi> = [\alpha, \beta]^T$

In general however, a quantum computer does not consist of one qubit but many qubits that interact with each other. The state of the quantum computer ($|\psi>$) is a combination of tese qubit states and hence a combination of the basis states of the n-qubit system.

The number of basis states of a system increases with the number of qubits as $2^n$ (n=number qubits) and hence the state of the quantum computer with n-qubits is fully described by a vector with $2^n$ entries (the probability amplitudes of the basis states).

For example for a system of 2-qubits each qubit can be in state |0> or |1>, this means that a 2-qubits system can  be measured in states |00>, |01>, |10> or |11> (so all possible permutations). These are the basis states. Before measuring, the 2-qubit system can be any combination of this basis states:

$|\psi> = \alpha|00> + \beta|01> +\gamma|10> +\delta|11>$ 

and can be fully represented by the vector $[\alpha, \beta,\gamma, \delta]^T$

Modifying a quanutm state means modifiying the probability amplitudes that describe it (the entries in the above vector). Measurments are a type of operations that do this. They "turn" all probability amplitudes to zero, except for one, which will have a value of 1.0 (basically they project the quantum state into one of the basis states). 
There are however many other operations that can modify the entries of quantum state vector and hence the quantum state of an n-qubit system. 
These operations (called quantum gates) can be used to manipulate the quantum state into a given outcome, hence we can use them to create algorithms. A quantum gate can be represented as $2^n$x$2^n$ unitary matrices (n the number of qubits in the system). The application of quantum gates to qubits corresponds to the multiplication of the n-qubit state with the quantum gate matrix and results in a modification of the $2^n$ entries of the quantum state vector. Some gates are comparable to classical gates. For example the classical NOT gate can be compared to the quantum X gate. In classical computing the NOT-gate converts a 0 into a 1 and a 1 into a 0. The quantum X-gate swaps the probability amplitudes of both states and converts $\alpha |0> + \beta|1>$ into $\beta|0> + \alpha|0>$, which basically means the probability of measureing 0 or 1 is swap.

For more examples of quantum gates we refer the reader to https://en.wikipedia.org/wiki/Quantum_logic_gate. For a more indepth knowledge of quantum gates: https://arxiv.org/pdf/1110.2998.pdf. How these gates are physically implemented depends on the type of qubits: trapped ions, NV centers, photons, etc. 

Most quantum gates act only in a small number of qubits (mostly due to experimental limitations). This means they are sparse matrices. This is not a limitation as there exist sets of one- and two-qubit quantum gates which are universal for quantum computation: any operation possible on a quantum computer can be reduced to this set of gates.



One classification of quantum gates is the Clifford vs non-Clifford gates(https://en.wikipedia.org/wiki/Clifford_gates). The Clifford gates are important, because a universal gate set consists of all the Clifford gates plus one non-Clifford gate. The Clifford-gates include the Pauli-gates (I, X, Y and Z), the Hadamard gate, the S gate and the controlled-X gate. The non-Clifford gate group is an infinite group with all the other quantum gates. Clifford gates are interesting for quantum simulators because they can be simualted in polynomial time, while non-cliford gates are exponentially hard to simulate on a classical computers (Gottesman-Knill theorem). For more information about the Gottesman-Knill theorem, reading "Improved Simulation of Stabilizer Circuits" by Scott Aaronson and Daniel Gottesman (Aaronson and Gottesman, 2004) is recommend.

### 1.2. Benchmarking quantum simulators<a id='quantumsim'></a>

A quantum simulator is a program that simulates how a quantum computer works; the exact calculation of the effects of every single gate on the qubits. Quantum simulators are used to increase and facilitate the development of quantum algorithms, their testing and debugging while non-trivial size quantum computer become available. They also allow for hardware-software co-design (Haner et al. https://arxiv.org/abs/1604.06460).
The need of quantum simulators has lead to the development of many different frameworks and libraries (for a reasonably complete list we refer the reader to https://www.quantiki.org/wiki/list-qc-simulators), so  which one should we use for a given problem? Beyond understanding the different capabilitites of the softwares it is important to understand their functionality and performance in order to chose the most appropiate one for a given application/problem.

Quantum simulators can differ from one an other on how the quantum state is stored, what gates are implemented, how measurments are performed, type of error sources that can be included, etc. These differences make it difficult to define a benchmark. 

In table below we present some of the differences among some libraries. If you want to look more into these different libraries Quantum Open Source Foundation https://qosf.org/ has a more complete list of them. 

|Library | explanation | link|
| :------ | :------------- | :---- |
| Cirq | Cirq is a Python library for writing, manipulating, and optimizing quantum circuits and running them against quantum computers and simulators. | https://github.com/quantumlib/Cirq | 
| PennyLane | A cross-platform Python library for quantum machine learning, automatic differentiation, and optimization of hybrid quantum-classical computations | https://pennylane.ai/ |
| ProjectQ | Powerful open source software for quantum computing | https://projectq.ch/ |
| QuEST | The Quantum Exact Simulation Toolkit is a high performance simulator of universal quantum circuits, state-vectors and density matrices. | https://github.com/quest-kit/QuEST |
| Quantum++ | Quantum++ is a modern C++11 general purpose quantum computing library, composed solely of template header files. | https://github.com/softwareQinc/qpp |
| Quantum Inspire | Run your own quantum algorithms on one of our simulators or hardware backends and experience the possibilities of quantum computing. | https://www.quantum-inspire.com/ |
| Qiskit | Qiskit is an open-source framework for working with quantum computers at the level of circuits, pulses, and algorithms. | https://qiskit.org/ |
| Qulacs | Qulacs is a python/C++ library for fast simulation of large, noisy, or parametric quantum circuits. | https://github.com/qulacs/qulacs |
| Strawberry Fields | Strawberry Fields is a full-stack Python library for designing, simulating, and optimizing continuous variable quantum optical circuits. | https://github.com/xanaduai/strawberryfields |

One of the main issues with quantum simulators is their limitations: they are normally memory bound and network bandwith limited.

As mentioned before, the size of the quantum states and quantum gates scales exponentially with the number of qubits. This also implies that the time and memory it takes to create them and operate with them in a classical computer in general scales exponentially, whcih limits the amount of qubits we can currently simulate in a computer. With this in mind, it is interesting to compare how the execution time and memory requirements scale with the number of qubits and depth of the circuit for the different quantum simulators, so basically how complex can the algorithms we want to simulate be. In previous papers (Jones et al., 2018) random circuits have been used to benchmark the performance of quantum simulators. Here, we will follow the same approach. 



### 1.3. Random circuits<a id='randomcircuits'></a>
As mentioned before, given a quantum system of n-qubits, there is a continuous set of possible states: all of the different combination of the $2n$ basis states (different values of the probability amplitudes). 
The capacity of exploring all states is (in part) where the quantum computer powers shine. Classically this could take many many computational hours, months, even years, but using the quantum properties fo the system this can be doen in principle relatively fast. An important question however is: how can we check we are exploring all sattes? One way of assesing this is by randomly producing n-qubit states and verifying that they are picked uniformly. 
This task although possible is also not straight forward for a quantum computer.
The reason behind this is that most of the states are what we call "entangled states", and entangled states are not only difficult to create but also highly unstable: they tend to decohere quickly due to interactions with an outside environment, which essentially drives the qubits to an unentangled state (like all qubits |0>) due to relaxation. 

Random quantum circuits are circuits that can generate quantum random states: when fed a constant input, they produce a random quantum state in each execution. Executing it many times yields a sample of quantum states that contains fresh randomness inherent to the quantum evaluation. Sampling the output distribution of a random  quantum circuit is a hard computational task and hence not possible with a classical computer. For the curious, this is basically what Google proved in their 2019 experiment of quantum supremacy.

In this tutorial we will implement and test different random circuits on two libraries: Qiskit and Qulacs and we will analyze their performance and properties.

###  1.4. Qiskit and Qulacs <a id='qulacs'></a>

Before going into the actual code, below some summary properteis of these simulators (for more info we refer the reader to the documentation pages of the software https://qiskit.org/documentation/, http://docs.qulacs.org/en/latest/guide/2.0_python_advanced.html#).

The default device/setting in Qiskit is:
* The qubit are stored in a QuantumRegister and the measurement of the qubits are stored in a "ClassicalRegister". 
* The QuantumRegister and ClassicalRegister make the QuantumCircuit.
* If we pass the entire quantum and classical registers as arguments to measure, the function will by default make a total measurement on the system, and store each qubit’s measurement results to the corresponding index in the ClassicalRegister: classical register numbers are backwards (based on the bit string  as usually defined in classical computer such that the LSB (least significant bit) is the right-most index)
* The QuantumCircuit object is just a list of instructions, which aren’t actually carried out until we run it on some simulator (backend)
* To execute the circuit on a certain backend you have to execute the whole circuit with that backend. That creates an object that handles a job, from the job you can extract the result.
* In the case of the 'qasm_simulator' the output will be how many time a certain number is measured, the circuit will be run 1024 times on default. An example of the default output you can get is \{'1': 519, '0': 505\}.
* In the case of the 'statevector_simulator', the circuit is executed a single shot and it returns the final quantum statevector of the simulation (wavefunction)
* Both these backends are still classical (not callign any quanutm devices)
* Qiskit Uses QASM simulator as quantum assembly.
* N-qubit Clifford operator is stored as a length 2N StabilizerTable using the convention from reference 

Qulacs is not a well known library, but it has some great features that are useful for researchers. One of the most important features are that it can run on a GPU for faster simulations, in this notebook we will only use qulacs on a CPU. It also claims to compress circuits to simulate faster than other libraries. 

The default device in Qulacs is:
* Quantum states allocates and manages 2n complex arrays on the CPU/GPU with the precision of complex128.
* Necessary memory is secured at the time of instance creation and released when the instance is destroyed
* All the gates are stored in a QuantumCircuit of a certain amount of qubits. 
* Quantum states have classical registers as arrays of integers with variable length. The classical register is used to write the result of the Instrument operation or to describe a gate that executes conditions as the result of the classical register. 
* Special gates are those that have a pre-specified gate matrix and can only perform limited deformations on quantum gates. For example, Pauli gate, rotation Pauli gate, projection measurement, etc. are supported. The advantage of a special gate is that the update function of the quantum state is more efficient than a general gate with limited properties. 
* Gates can be defined as dense or sparse matrices
* With a lot of qubits you will get an out of memory error, because Qulacs is not memory efficient. 
* You need a separate QuantumState and this state has to be set to the one or zero state for measurement.
* QASM is used as it's quantum assembly.
* To execute the circuit on the quantum state you have to update that state with the circuit.
* It measures a vector of the quantum state.
* By combining quantum gates into a single quantum gate, the number of quantum gates can be reduced and the time required for numerical calculations can be reduced. (Of course, if the number of target qubits increases, or if a quantum gate with a dedicated function is synthesized into a quantum gate without a dedicated function, the total calculation time may not decrease. It depends.)



Every library applies gates differently, Qulacs puts the gates in a queue that will be executed after calling all the gates, while Qiskit applies the gates immediately and measures the qubit at the end. Another difference is that not all libraries measure with the same output. Qiskit can measure a 1 or a 0 (or the statevector); Qulacs measures a state vector. We have to keep these differences in mind while comparing these libraries with each other. Every library also differs in the gate set available, in the table below we present this gates per librabry:


| Quantum simulators | Language | Can run on | Single-Qubit Gates | Multiqubit Gates|
| :----------------- |:---------| :------------- | ------------------ | --------------- |
| Qiskit | Python | CPU | X, Y, Z, H, S, S†, T, T†, Rx, Ry, Rz, U1, U2, U3 | CNOT, CZ, CY, SWAP, Toffoli, CSWAP 
| Qulacs | Python/C++ | CPU or GPU | I, X, Y, Z, H, S, S†, T, T†,sqrtX, sqrtX†, sqrtY, sqrtY†, P0, P1, RX, RY, RZ, U1, U2, U3 | CNOT, CZ, SWAP |


## 2. Benchmark examples <a id ='examples'></a>

In this section we will look at three different benchmarks. Lets first import all the neccesary libraries:


In [ ]:
# the quantum libraries import 
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer, IBMQ
from qulacs import Observable, QuantumState, QuantumCircuit as QC

# to measure the time and memory it takes
import time
import tracemalloc
import resource

# imports to plot the graphs
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import mark_inset, inset_axes
from qiskit.visualization import plot_histogram
import numpy as np


### 2.1. Hadamards <a id='hadamards'></a>

Random circuits in general contain Hadamard gates and some sort(s) of "entangling" gates (2 or more qubit controll gates like the CNOT) therefore it is important to understand what these gates do and also benchmark them individually. 

Let's start first with a just setting up a circuit:  we define the quantum register of one qubit and the classical register (to store the measurments of this qubit). These two registers together form the circuit:


In [ ]:
# defining the device 
q = QuantumRegister(1, 'q')
c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(q, c)

circuit.measure(q, c)

counts = execute(circuit, Aer.get_backend('qasm_simulator')).result().get_counts()
plot_histogram(counts)



The qubit is inititalized in state $|\psi> = 1.0 |0> +0.0|1>$, the probability of measuring state |1> is zero. In this example we have used the 'qsam_simulator' backend which means that the circuit is build and measured 1024 times. In all runs, when we measure the state of the qubt we get state |0> (as expected). Below how the circuit looks like:



In [ ]:
#Here how the circuit look like:
circuit.draw(output='mpl')

Now lets apply a Hadamard gate before measuring:

In [ ]:
# defining the device 
circuit = ()
q = QuantumRegister(1, 'q')
c = ClassicalRegister(1, 'c')
circuitH = QuantumCircuit(q, c)

# A hadamard operation and measurement
circuitH.h(q)
circuitH.measure(q, c)

# plot the histogram with probabilities of measuring a one or a zero.
counts = execute(circuitH, Aer.get_backend('qasm_simulator')).result().get_counts()
job = execute(circuitH, Aer.get_backend('qasm_simulator'))
plot_histogram(counts)


In [ ]:
#Here how the circuit look like:
circuitH.draw(output='mpl')

Now the probability of measuring $|0>$ or $|1>$ is more or less evenly distributed. The Hadamard gate "distributes" the probability of been in a given state among the single qubit basis states, it creates a superposition state. From the 1024 times we build and measured the system about 50% of the times we measured state |0> and 50% of the times we measured state |1>.

The Hadamard gate is a single qubit gate, so what happens if we apply it to a 2-qubit system?

In [ ]:
# defining the device 
q = QuantumRegister(2, 'q')
c = ClassicalRegister(2, 'c')
circuitH2 = QuantumCircuit(q, c)

# A hadamard operation on qubit 0 and measure the circuit
circuitH2.h(q)
circuitH2.measure(q, c)

# draw the circuit
circuitH2.draw(output='mpl')

# plot the histogram with probabilities of measuring a one or a zero.
counts = execute(circuitH2, Aer.get_backend('qasm_simulator')).result().get_counts()
plot_histogram(counts)

In [ ]:
circuitH2.draw(output='mpl')

The application of a Hadamard gate to each of the qubits "distributes" the evenly the probability of measuring each state. Each qubit can be in either 0 or 1, so there are 4 possible states (|00>, |01>, |10>, |11>).

### 2.2. CNOT <a id='cnot'></a>
The CNOT is a two qubit controll gate very much used to ensure entanglement. Let's see some examples:

In [ ]:
# defining the device 
q = QuantumRegister(2, 'q')
c = ClassicalRegister(2, 'c')
circuitCNOT = QuantumCircuit(q, c)

# A cnot operation: q0 is the controll qubit
circuitCNOT.cnot(0,1)
circuitCNOT.measure(q,c)

# plot the histogram with probabilities of measuring a one or a zero.
counts = execute(circuitCNOT, Aer.get_backend('qasm_simulator')).result().get_counts()
plot_histogram(counts)

In [ ]:
circuitCNOT.draw(output='mpl')

The CNOT gate (or controlled X) flips the second qubit (the target qubit) if and only if the first qubit (the control qubit) is |1>. In this case the control qubit is |0> so no flip. We can change the control qubit from |0> to |1> using an X gate and check again:

In [ ]:
# defining the device 
q = QuantumRegister(2, 'q')
c = ClassicalRegister(2, 'c')
circuitXCNOT = QuantumCircuit(q, c)

# A cnot operation: q0 is the controll qubit
circuitXCNOT.x(0)
circuitXCNOT.cnot(0,1)
circuitXCNOT.measure(q,c)

# plot the histogram with probabilities of measuring a one or a zero.
counts = execute(circuitXCNOT, Aer.get_backend('qasm_simulator')).result().get_counts()
plot_histogram(counts)

The X gate flips q0 to |1> and because the control qubit is now |1>,  the CNOT gate flips the target qubit from |0> to |1>. Therefore, the final state is always |11>. 

Now lets apply a Hadamard on the control qubit followed by a CNOT gate:

In [ ]:
# defining the device 
q = QuantumRegister(2, 'q')
c = ClassicalRegister(2, 'c')
circuitHCNOT = QuantumCircuit(q, c)


circuitHCNOT.h(0)
circuitHCNOT.cnot(0,1)
circuitHCNOT.measure(q,c)

# plot the histogram with probabilities of measuring a one or a zero.
counts = execute(circuitHCNOT, Aer.get_backend('qasm_simulator')).result().get_counts()
plot_histogram(counts)

In [ ]:
circuitHCNOT.draw(output='mpl')

The combination of a Hadamard and a CNOT give us the state vector $|\psi> = 1/ \sqrt(2)|00> + 1\sqrt(2)|11>$.
There is an equal chance of meassuring |00> and |11>. The state created is one of the so called "Bell states" (https://en.wikipedia.org/wiki/Bell_state). The Bell states are entangled, that means that measuring one already "defines" the value of the other. If we measure q0 at state |0>, then we know q1 is also in state |0>, the same if we measure q0 in |1>. They are somehow linked! This link is called entanglement: the state of the system can not be described by the individual qubits, only by the whole multi-qubit register (https://en.wikipedia.org/wiki/Quantum_entanglement). To create the other three Bell states of 2-qubits you need a different "starting" state, for example start with q0 in |1> and q1 in |1>,

In [ ]:
# defining the device 
q = QuantumRegister(2, 'q')
c = ClassicalRegister(2, 'c')
circuitXHCNOT = QuantumCircuit(q, c)

circuitXHCNOT.x(0)
circuitXHCNOT.x(1)
circuitXHCNOT.h(0)
circuitXHCNOT.cnot(0,1)
circuitXHCNOT.measure(q,c)

# plot the histogram with probabilities of measuring a one or a zero.
counts = execute(circuitXHCNOT, Aer.get_backend('qasm_simulator')).result().get_counts()
plot_histogram(counts)

In [ ]:
circuitXHCNOT.draw(output='mpl')

In this case, if we measure q0 at |0> we know q1 is |1> and the other way around. Bell states are a maximally entangled states. 

We can use the same technique to create an entangled state of 3 qubits:

In [ ]:
# defining the device 
q = QuantumRegister(3, 'q')
c = ClassicalRegister(3, 'c')
circuitXHCNOT = QuantumCircuit(q, c)

circuitXHCNOT.h(0)
circuitXHCNOT.cnot(0,1)
circuitXHCNOT.cnot(0,2)
circuitXHCNOT.measure(q,c)

# plot the histogram with probabilities of measuring a one or a zero.
counts = execute(circuitXHCNOT, Aer.get_backend('qasm_simulator')).result().get_counts()
plot_histogram(counts)

Again, if we only measure qubit q0, we already know the state of the other two.

### 2.3. Timing gates:  Hadamard, CNOT, T-gate <a id='timing'></a>
We would like to understand how the simualtion time of these gates scales with the number of qubits. Based on our knowledge of the simualtions limitations (memory bound), we will chose a maximum of 67 qubits. Below we define a code that computes the average time a circuit in n-qubits takes to run. Every circuit is run 20 times.



Lets start by timing the Hadamard gate. Each circuit will be run 20 times. Below the code for Qiskit:

In [ ]:
# Amount of qubits we are going to simulate
qisQubitsH = 40

# Array for the results to plot them afterwards
qisHadamards = np.array([], dtype=float)

# global variables 
times = 1


for p in range(2, qisQubitsH):
    tt_tot = 0.0

    for t in range(times):
        
        # defining the device 
        q = QuantumRegister(p, 'q')
        c = ClassicalRegister(p, 'c')
        circ = QuantumCircuit(q, c)
        t_start = time.perf_counter()
        
        # A hadamard operation on every qubit
        circ.h(q)

        circ.measure(q, c)

        # the default backend
        backend = Aer.get_backend('statevector_simulator')
        job_sim = execute(circ, backend)

        #results = job_sim.result()

        t_stop = time.perf_counter()
        t_tot = t_stop - t_start

        tt_tot += t_tot
        del q
        del c
        del circ
        del job_sim

    t_avg = tt_tot / times
    qisHadamards = np.append(qisHadamards, [t_avg])

print(qisHadamards)

Important to do measure? otherwise ram stays full 
.. processors stay full on? why?

In [ ]:
# Amount of qubits we are going to simulate
qisQubitsH = 10

# Array for the results to plot them afterwards
qisHadamards2 = np.array([], dtype=float)

# global variables 
times = 1


for p in range(2, qisQubitsH):
    tt_tot = 0.0

    for t in range(times):
        
        # defining the device 
        q = QuantumRegister(p, 'q')
        c = ClassicalRegister(p, 'c')
        circ = QuantumCircuit(q, c)
        t_start = time.perf_counter()
        
        # A hadamard operation on every qubit
        #circ.h(q)

        #circ.measure(q, c)

        # the default backend
        backend = Aer.get_backend('statevector_simulator')
        job_sim = execute(circ, backend)

        #results = job_sim.result()

        t_stop = time.perf_counter()
        t_tot = t_stop - t_start

        tt_tot += t_tot
        del q
        del c
        del circ
        del job_sim

    t_avg = tt_tot / times
    qisHadamards2 = np.append(qisHadamards2, [t_avg])

print(qisHadamards2)

In [ ]:
# Amount of qubits we are going to simulate
qisQubitsH = 60

# Array for the results to plot them afterwards
qisHadamards3 = np.array([], dtype=float)

# global variables 
times = 10


for p in range(2, qisQubitsH):
    tt_tot = 0.0

    for t in range(times):
        
        # defining the device 
        q = QuantumRegister(p, 'q')
        c = ClassicalRegister(p, 'c')
        circ = QuantumCircuit(q, c)
        t_start = time.perf_counter()
        
        # A hadamard operation on every qubit
        circ.h(q)

        circ.measure(q, c)

        # the default backend
        backend = Aer.get_backend('statevector_simulator')
        job_sim = execute(circ, backend)

        #results = job_sim.result()

        t_stop = time.perf_counter()
        t_tot = t_stop - t_start

        tt_tot += t_tot
        del q
        del c
        del circ
        del job_sim

    t_avg = tt_tot / times
    qisHadamards3 = np.append(qisHadamards3, [t_avg])

print(qisHadamards3)

In [ ]:
#circEx.draw(output='mpl')

The code for the hadamards circuits in Qulacs:

In [ ]:
# Amount of qubits we are going to simulate
qulQubitsH = 22
times = 10

# Array for the results to plot them afterwards
qulHadamards = np.array([], dtype=float)

for p in range(2, qulQubitsH):
    tt_tot = 0.0
    for t in range(times):
       

        state = QuantumState(p)
        state.set_zero_state()

        circuit = QC(p)
        t_start = time.perf_counter()

        for i in range(p):
            circuit.add_H_gate(i)

        circuit.update_quantum_state(state)
        #output = state
        del circuit
        del state

        t_stop = time.perf_counter()
        t_tot = t_stop - t_start

        tt_tot += t_tot

    t_avg = tt_tot / times
    qulHadamards = np.append(qulHadamards, [t_avg])

print(qulHadamards)

Some plotting adjustments for the ranges:

In [ ]:
y1 = []
if qisQubitsH < qulQubitsH: 
    for i in range(qulQubitsH - qisQubitsH): 
        qisHadamards = np.append(qisHadamards, [np.nan])
    y1 = range(2, qulQubitsH)
elif qisQubitsH > qulQubitsH:
    for i in range(qisQubitsH - qulQubitsH): 
        qulHadamards = np.append(qulHadamards, [np.nan])
    y1 = range(2, qisQubitsH)
else:
    y1 = range(2, qisQubitsH)

And finally the plotting: 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=150) 
y1 = range(2, qisQubitsH)
# the whole graph
ax.plot(y1, qisHadamards, 'c', label='Qiskit', linewidth=2, marker='^', markersize=4)
ax.plot(y1, qisHadamards2, 'c', label='Qiskit circuit', linewidth=2, marker='*', markersize=4)
ax.plot(y1, qisHadamards3, 'c', label='Qiskit full', linewidth=2, marker='', markersize=4)

#ax.plot(y1, qulHadamards, 'm', label='Qulacs', linewidth=2, marker='x', markersize=4)
plt.xticks(np.arange(2, qisQubitsH, step=4))
plt.title('Hadamards', fontsize=14)

plt.xlabel('$n$ qubits', fontsize=14)
plt.ylabel('time in seconds', fontsize=14)
ax.legend()

# zoomed in part
#axins = inset_axes(ax, width="40%", height=1., loc='center right', borderpad=2)
#mark_inset(ax, axins, loc1=3, loc2=4, fc='none', ec='0.5')

#axins.plot(y1, qisHadamards, 'c', label='Qiskit', linewidth=2, marker='^', markersize=4)
#axins.plot(y1, qulHadamards, 'm', label='Qulacs', linewidth=2, marker='x', markersize=4)

#ax.set_xlim(2, 10)
#ax.set_ylim(-0.01, 0.1)
#plt.xticks(np.arange(2, 66, step=10))

# show the whole graph
plt.show()

The Hadamard is a single qubit gate and is part of the Cliford gates.
Therefore we expect the computational time to increase polynomial with time if the the Gottesman-Knill theorem is implemented. Qiskit makes use of the Gottesman-Knill theorem to simulate the Clifford gates efficiently and therefore we see a linear increase in simualtion time with number of qubits. Qulacs apparently not.
In Qulacs we see an exponential grow in time after about 18 qubits.

 -Why does Qulacs performs better before the exponential regime? (Qulacs running on CPU) Memory allocation?
 How states are allocated or the gates?
- Time is not considering the circuit allocation
- Why re there some peaks in Qiskit? They do get reduced with the amoutn of runs, but seems weird and inconsistent
- is this better compared using the statevector backend of qiskit? Alos qsikit linear
Lets now take a look at the CNOT

In [ ]:
# Amount of qubits we are going to simulate
qisQubitsCNOT = 67
times = 20
tt_tot_array = np.array([], dtype=float)
# Array for the results to plot them afterwards
qisCNOT = np.array([], dtype=float)
for p in range(2, qisQubitsCNOT):
    tt_tot = 0.0
      

        # defining the device 
    q = QuantumRegister(p, 'q')
    c = ClassicalRegister(p, 'c')
    circ = QuantumCircuit(q, c)
    circ.h(0)
    for t in range(times):
        t_start = time.perf_counter()
        
        for i in (range(1, p)): 
            circ.cnot(0, i)
                    
        circ.measure(q, c)

        # the default backend
        backend = Aer.get_backend('qasm_simulator')
        job_sim = execute(circ, backend)

        results = job_sim.result()

        t_stop = time.perf_counter()
        t_tot = t_stop - t_start

        tt_tot += t_tot
        tt_tot_array = np.append(tt_tot_array, tt_tot)
        
    t_avg = tt_tot / times
    #t_error = np.sum(tt_tot_array)/times
    
    qisCNOT= np.append(qisCNOT, [t_avg])

print(qisCNOT)

In [ ]:
# Amount of qubits we are going to simulate
qulQubitsH = 22
times = 20

# Array for the results to plot them afterwards
qulHadamards = np.array([], dtype=float)

for p in range(2, qulQubitsH):
    tt_tot = 0.0
    for t in range(times):
       

        state = QuantumState(p)
        state.set_zero_state()

        circuit = QC(p)
        t_start = time.perf_counter()

        for i in range(p):
            circuit.add_CNOT_gate(0,i)

        circuit.update_quantum_state(state)
        output = state
        del circuit
        del state

        t_stop = time.perf_counter()
        t_tot = t_stop - t_start

        tt_tot += t_tot

    t_avg = tt_tot / times
    qulHadamards = np.append(qulHadamards, [t_avg])

print(qulHadamards)

In [ ]:
y1 = range(2, qisQubitsCNOT)
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=150) 

# the whole graph
ax.plot(y1, qisCNOT, 'c', label='Qiskit', linewidth=2, marker='^', markersize=4)
#ax.plot(y1, qulHadamards, 'm', label='Qulacs', linewidth=2, marker='x', markersize=4)
#plt.xticks(np.arange(2, qisQubitsCNOT, step=4))
plt.title('CNOT', fontsize=14)

plt.xlabel('$n$ qubits', fontsize=14)
plt.ylabel('time in seconds', fontsize=14)
ax.legend()

# zoomed in part
#axins = inset_axes(ax, width="40%", height=1., loc='center right', borderpad=2)
#mark_inset(ax, axins, loc1=3, loc2=4, fc='none', ec='0.5')

#axins.plot(y1, qisCNOT, 'c', label='Qiskit', linewidth=2, marker='^', markersize=4)
#axins.plot(y1, qulHadamards, 'm', label='Qulacs', linewidth=2, marker='x', markersize=4)

#axins.set_xlim(2, qisQubitsCNOT)
#axins.set_ylim(-0.01, 0.1)
#plt.xticks(np.arange(2, qisQubitsCNOT, step=5))

# show the whole graph
plt.show()

Again, Qiskit timing seems to increase linearly (the CNOT is also a Clifford gate).
TODO: does the circuit need to be relesed? Whay otherwise are those little picks there?

To visualize the maximally entangled state created, and final circuit uncomment the lines below:

In [ ]:
# plot the histogram with probabilities of measuring a one or a zero.
#counts = execute(circ, Aer.get_backend('qasm_simulator')).result().get_counts()
#plot_histogram(counts)

In [ ]:
#circ.draw(output='mpl')

Finally, lets analyze a non-Clifford gate to see the difference: the T gate.
    

In [ ]:
# Amount of qubits we are going to simulate
qisQubitsT = 20
times = 20

# Array for the results to plot them afterwards
qisT = np.array([], dtype=float)
for p in range(2, qisQubitsT):
    tt_tot = 0.0
    for t in range(times):
        t_start = time.perf_counter()

        # defining the device 
        q = QuantumRegister(p, 'q')
        c = ClassicalRegister(p, 'c')
        circT = QuantumCircuit(q, c)
        circT.t(q)
                            
        circT.measure(q, c)

        # the default backend
        backend = Aer.get_backend('qasm_simulator')
        job_sim = execute(circT, backend)

        results = job_sim.result()

        t_stop = time.perf_counter()
        t_tot = t_stop - t_start

        tt_tot += t_tot

    t_avg = tt_tot / times
    qisT= np.append(qisT, [t_avg])

print(qisT)

In [ ]:
y1 = range(2, qisQubitsT)
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=150) 

# the whole graph
ax.plot(y1, qisT, 'c', label='Qiskit', linewidth=2, marker='^', markersize=4)
#ax.plot(y1, qulHadamards, 'm', label='Qulacs', linewidth=2, marker='x', markersize=4)
#plt.xticks(np.arange(2, qisQubitsCNOT, step=4))
plt.title('T-gate', fontsize=14)

plt.xlabel('$n$ qubits', fontsize=14)
plt.ylabel('time in seconds', fontsize=14)
ax.legend()

# zoomed in part
#axins = inset_axes(ax, width="40%", height=1., loc='center right', borderpad=2)
#mark_inset(ax, axins, loc1=3, loc2=4, fc='none', ec='0.5')

#axins.plot(y1, qisCNOT, 'c', label='Qiskit', linewidth=2, marker='^', markersize=4)
#axins.plot(y1, qulHadamards, 'm', label='Qulacs', linewidth=2, marker='x', markersize=4)

#axins.set_xlim(2, qisQubitsCNOT)
#axins.set_ylim(-0.01, 0.1)
#plt.xticks(np.arange(2, qisQubitsCNOT, step=5))

# show the whole graph
plt.show()

### 2.2. Random circuit implementation 1: time <a id='RC1time'></a>

A random circuit contains  a  random  sequence  of  gates. We  generate  our  random  circuits by the algorithm in (https://arxiv.org/abs/1608.00263), which fixes the topology for a given depth and number of qubits, though randomises the sequence of single qubit gates. The total number of gates goes like O(n*d), d=depth of circuit and the ratio between single and controll gates is fixed to 1.2 $\pm$ 0.2

TODO: read paper carefully!!! Is there a ratio between clifford and non Clifford or only using CLifford ones?

The first random circuit implementation we will consider consist of randomly applying a Hadamard, T-gate, X-gate, Y-gate or Controlled Z-gate (Jones et al., 2018). As a brief background, if we think of the probability amplitudes of a single qubit as a vector in a 3D sphere, all single qubit gates modify the vector by rotating it in the sphere. The X gate rotates it 180 degrees. The T and Y- gates also rotate the vector but in different ways.
The Y- gate is also part of the Clifford gates and can be efficiently simualted by Qiskit.

The Controlled Z (CZ) is a two-qubit gate and is the controlled version of the Z gate (same as CNOT being the controlled version of the X gate). This gate applies a Z operation (change of sign in the |1> state) on the target qubit when the control is |1>. It can also be described as a conditional operation that performs a sign shift only when the two qubit state is |11>.

In the code below you will see that we will run the random circuit up  to 29 qubits (we already know the limitations):

In [ ]:
# amount of qubits we are going to run
depth = 20 
width = 6 #example
qisQubitsRC1 = 27
qulQubitsRC1 = 28

qisRC1t = np.array([], dtype=float)
qulRC1t = np.array([], dtype=float)

# array with every possible gate for this implementation 
arr = ['H', 'T', 'X', 'Y', 'C(Z)']

Below is an example of this implementation of a random circuit with 6 qubits.

TODO: Is the ratio of single vs controlled fixed? Is the number fixed? Shoudl there be also a ratio cliffor non clifford?


In [ ]:
# defining the device 
q = QuantumRegister(width, 'q')
c = ClassicalRegister(width, 'c')
circR1 = QuantumCircuit(q, c)

# Randomly applying the random circuit gate set
for i in range(depth):
    amount = np.random.randint(0, 3)
    for j in range(amount):
        # chosing a random gate and applying it on a random circuit
        rGate = np.random.choice(arr)
        i0 = np.random.randint(0, width)
        i1 = np.random.randint(0, width)
        while i0 == i1: i1 = np.random.randint(0, width)

        if rGate == 'H':
            circR1.h(q[i0])
        elif rGate == 'T':
            circR1.t(q[i0])
        elif rGate == 'X':
            circR1.x(q[i0])
        elif rGate == 'Y':
            circR1.y(q[i0])
        elif rGate == 'C(Z)':
            circR1.cz(q[i0], q[i1])

circR1.barrier()
circR1.measure(q, c)

# draw the circuit
circR1.draw(output='mpl')

The code for measuring the time it takes to run this random circuit implementation in Qiskit:

In [ ]:
for k in range(2, qisQubitsRC1):
    tt_tot = 0.0
    for t in range(times):
        t_start = time.perf_counter()

        # defining the device
        q = QuantumRegister(k, 'q')
        c = ClassicalRegister(k, 'c')
        circR1 = QuantumCircuit(q, c)

        for i in range(depth):
            amount = np.random.randint(0, 3)
            for j in range(amount):
                # chosing a random gate and applying it on a random circuit
                rGate = np.random.choice(arr)
                i0 = np.random.randint(0, k)
                i1 = np.random.randint(0, k)
                while i0 == i1: i1 = np.random.randint(0, k)

                if rGate == 'H':
                    circR1.h(q[i0])
                elif rGate == 'T':
                    circR1.t(q[i0])
                elif rGate == 'X':
                    circR1.x(q[i0])
                elif rGate == 'Y':
                    circR1.y(q[i0])
                elif rGate == 'C(Z)':
                    circR1.cz(q[i0], q[i1])

        circR1.measure(q, c)

        # the default backend to run the whole circuit
        backend = Aer.get_backend('qasm_simulator')
        job_sim = execute(circR1, backend)
        result = job_sim.result()

        t_stop = time.perf_counter()
        t_tot = t_stop - t_start

        tt_tot += t_tot

    t_avg = tt_tot / times

    qisRC1t = np.append(qisRC1t, [t_avg])

print(qisRC1t)

The code for measuring the time it takes to run this random circuit implementation in Qulacs:

In [ ]:
for k in range(2, qulQubitsRC1):
    tt_tot = 0.0
    for t in range(times):
        t_start = time.perf_counter()

        state = QuantumState(k)
        state.set_zero_state()

        circuit = QC(k)

        for i in range(depth):
            amount = np.random.randint(0, 3)
            for j in range(amount):
                rGate = np.random.choice(arr)
                i0 = np.random.randint(0, k)
                i1 = np.random.randint(0, k)
                while i0 == i1: i1 = np.random.randint(0, k)

                if rGate == 'H':
                    circuit.add_H_gate(i0)
                elif rGate == 'T':
                    circuit.add_T_gate(i0)
                elif rGate == 'X':
                    circuit.add_X_gate(i0)
                elif rGate == 'Y':
                    circuit.add_Y_gate(i0)
                elif rGate == 'C(Z)':
                    circuit.add_CZ_gate(i0, i1)

        circuit.update_quantum_state(state)
        output = state
        del circuit
        del state

        t_stop = time.perf_counter()
        t_tot = t_stop - t_start

        tt_tot += t_tot

    t_avg = tt_tot / times
    qulRC1t = np.append(qulRC1t, [t_avg])

print(qulRC1t)

This is so you can play with the amount of qubits and still can plot the graph without problems. 

In [ ]:
y2 = []
if qisQubitsRC1 < qulQubitsRC1: 
    for i in range(qulQubitsRC1 - qisQubitsRC1): 
        qisRC1t = np.append(qisRC1t, [np.nan])
    y2 = range(2, qulQubitsRC1)
elif qisQubitsRC1 > qulQubitsRC1:
    for i in range(qisQubitsRC1 - qulQubitsRC1): 
        qulRC1t = np.append(qulRC1t, [np.nan])
    y2 = range(2, qisQubitsRC1)
else:
    y2 = range(2, qisQubitsRC1)

Below we will put these results in a graph in which you can see the time it takes to execute the random circuits on the increasing amount of qubits. 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=150) 

# the whole graph
ax.plot(y2, qisRC1t, 'c', label='Qiskit', linewidth=2, marker='^', markersize=4)
ax.plot(y2, qulRC1t, 'm', label='Qulacs', linewidth=2, marker='x', markersize=4)
plt.xticks(np.arange(2, qisQubitsRC1, step=2))
plt.title('Random Circuit 1', fontsize=14)

plt.xlabel('$n$ qubits', fontsize=14)
plt.ylabel('time in seconds', fontsize=14)
ax.legend()

# zoomed in part
axins = inset_axes(ax, width="40%", height=1., loc='center left', borderpad=2)
mark_inset(ax, axins, loc1=3, loc2=4, fc='none', ec='0.5')

axins.plot(y2, qisRC1t, 'c', label='Qiskit', linewidth=2, marker='^', markersize=4)
axins.plot(y2, qulRC1t, 'm', label='Qulacs', linewidth=2, marker='x', markersize=4)

axins.set_xlim(2, 21)
axins.set_ylim(-0.01, 0.1)
plt.xticks(np.arange(2, 22, step=2))

# show the whole graph
plt.show()

In this graph Qulacs is the most efficient. In this case Qiskit can not simulate 67 qubits, this is because the T- and the CZ-gate are not Clifford gates, their simulation scales exponentially! From this result we can conclued that the Clifford gates are only gates that are optimized in Qiskit.

### 2.3. Random circuit  implementation 1 memory <a id='RC1mem'></a>

In this part we will look at the memory it takes to execute the random circuit implementation 1. We will look at the maximum resident set size. With resident set size we mean the portion of memory occupied by a process that is held in main memory (RAM).

In [ ]:
qisRC1m = np.array([], dtype=float)
qulRC1m = np.array([], dtype=float)

The code for this random circuit implementation in Qiskit:

In [ ]:
for k in range(2, qisQubitsRC1):
    rss_tot = 0.0
    for t in range(times):
        tracemalloc.start()
        q = QuantumRegister(k, 'q')
        c = ClassicalRegister(k, 'c')
        circ = QuantumCircuit(q, c)

        for i in range(depth):
            amount = np.random.randint(0, 3)
            for j in range(amount):
                # chosing a random gate and applying it on a random circuit
                rGate = np.random.choice(arr)
                i0 = np.random.randint(0, k)
                i1 = np.random.randint(0, k)
                while i0 == i1: i1 = np.random.randint(0, k)

                if rGate == 'H':
                    circ.h(q[i0])
                elif rGate == 'T':
                    circ.t(q[i0])
                elif rGate == 'X':
                    circ.x(q[i0])
                elif rGate == 'Y':
                    circ.y(q[i0])
                elif rGate == 'C(Z)':
                    circ.cz(q[i0], q[i1])

        circ.measure(q, c)

        backend = Aer.get_backend('qasm_simulator')
        job_sim = execute(circ, backend)
        sim_result = job_sim.result()

        snapshot = tracemalloc.take_snapshot()
        top_stats = snapshot.statistics('lineno')
        tracemalloc.stop()
        max_rss = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

        rss_tot += max_rss

    rss_avg = rss_tot / times

    qisRC1m = np.append(qisRC1m, [rss_avg])

print(qisRC1m)

The code for this random circuit implementation in Qulacs:

In [ ]:
for k in range(2, qulQubitsRC1):
    rss_tot = 0.0
    for t in range(times):
        tracemalloc.start()

        state = QuantumState(k)
        state.set_zero_state()

        circuit = QC(k)

        for i in range(depth):
            amount = np.random.randint(0, 3)
            for j in range(amount):
                rGate = np.random.choice(arr)
                i0 = np.random.randint(0, k)
                i1 = np.random.randint(0, k)
                while i0 == i1: i1 = np.random.randint(0, k)

                if rGate == 'H':
                    circuit.add_H_gate(i0)
                elif rGate == 'T':
                    circuit.add_T_gate(i0)
                elif rGate == 'X':
                    circuit.add_X_gate(i0)
                elif rGate == 'Y':
                    circuit.add_Y_gate(i0)
                elif rGate == 'C(Z)':
                    circuit.add_CZ_gate(i0, i1)

        circuit.update_quantum_state(state)
        output = state
        del circuit
        del state

        snapshot = tracemalloc.take_snapshot()
        top_stats = snapshot.statistics('lineno')
        tracemalloc.stop()
        max_rss = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

        rss_tot += max_rss

    rss_avg = rss_tot / times
    qulRC1m = np.append(qulRC1m, [rss_avg])

print(qulRC1m)

This is so you can play with the amount of qubits and still can plot the graph without problems. 

In [ ]:
y3 = []
if qisQubitsRC1 < qulQubitsRC1: 
    for i in range(qulQubitsRC1 - qisQubitsRC1): 
        qisRC1m = np.append(qisRC1m, [np.nan])
    y3 = range(2, qulQubitsRC1)
elif qisQubitsRC1 > qulQubitsRC1:
    for i in range(qisQubitsRC1 - qulQubitsRC1): 
        qulRC1m = np.append(qulRC1m, [np.nan])
    y3 = range(2, qisQubitsRC1)
else:
    y3 = range(2, qisQubitsRC1)

Below we will put these results in a graph in which you can see the maximum resident set size it takes to execute the random circuits on the increasing amount of qubits.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=150) 

# the whole graph
ax.plot(y3, qisRC1m, 'c', label='Qiskit', linewidth=2, marker='^', markersize=4)
ax.plot(y3, qulRC1m, 'm', label='Qulacs', linewidth=2, marker='x', markersize=4)
plt.xticks(np.arange(2, qisQubitsRC1, step=2))
plt.title('Memory Clifford gates', fontsize=14)

plt.xlabel('$n$ qubits', fontsize=14)
plt.ylabel('maximum resident set size', fontsize=14)
ax.legend()

# show the whole graph
plt.show()

From this graph we can see that Qiskit uses the same maximun resident size as Qulacs.

### 2.4. Random circuit implementation 2 (clifford gates): time <a id='RCcliftime'></a>

Now lets build random circuits using a Universal set of only Clifford gates. If the Gottesman-Knill theorem
is correct then we will get similar results as the only Hadamard operations benchmark. 

In [ ]:
# the amount of qubits we will run
qisQubitsClif = 67
qulQubitsClif = 28

qisClifT = np.array([], dtype=float)
qulClifT = np.array([], dtype=float)

# an array with every Clifford gate
clifford = ['H', 'S', 'X', 'Y', 'Z', 'CX']

Below is an example of randomly appying the Clifford gates 6 qubits.

In [ ]:
# defining the device 
q = QuantumRegister(width, 'q')
c = ClassicalRegister(width, 'c')
circ = QuantumCircuit(q, c)

# Randomly applying the Clifford gates
for i in range(depth):
    amount = np.random.randint(0, 3)

    for i in range(amount):
        # chosing a random Clifford gate
        rGate = np.random.choice(clifford)
        i0 = np.random.randint(0, width)
        i1 = np.random.randint(0, width)
        while i0 == i1: i1 = np.random.randint(0, width)

        if rGate == 'H':
            circ.h(q[i0])
        elif rGate == 'S':
            circ.s(q[i0])
        elif rGate == 'X':
            circ.x(q[i0])
        elif rGate == 'Y':
            circ.y(q[i0])
        elif rGate == 'Z':
            circ.z(q[i0])
        elif rGate == 'CX':
            circ.cx(q[i0], q[i1])

circ.barrier()
circ.measure(q, c)

# draw the circuit
circ.draw(output='mpl')

The code for this random circuit implementation in Qiskit:

In [ ]:
for a in range(2, qisQubitsClif):
    tt_tot = 0.0
    for t in range(times):
        t_start = time.perf_counter()
        q = QuantumRegister(a, 'q')
        c = ClassicalRegister(a, 'c')
        circ = QuantumCircuit(q, c)

        for i in range(depth):
            amount = np.random.randint(0, 3)

            for i in range(amount):
                # chosing a random Clifford gate
                rGate = np.random.choice(clifford)
                i0 = np.random.randint(0, a)
                i1 = np.random.randint(0, a)
                while i0 == i1: i1 = np.random.randint(0, a)

                if rGate == 'H':
                    circ.h(q[i0])
                elif rGate == 'S':
                    circ.s(q[i0])
                elif rGate == 'X':
                    circ.x(q[i0])
                elif rGate == 'Y':
                    circ.y(q[i0])
                elif rGate == 'Z':
                    circ.z(q[i0])
                elif rGate == 'CX':
                    circ.cx(q[i0], q[i1])

        circ.measure(q, c)

        backend = Aer.get_backend('qasm_simulator')
        job_sim = execute(circ, backend)
        sim_result = job_sim.result()

        t_stop = time.perf_counter()
        t_tot = t_stop - t_start
        tt_tot += t_tot

    t_avg = tt_tot / times
    qisClifT = np.append(qisClifT, [t_avg])

print(qisClifT)

The code for this random circuit implementation in Qulacs:

In [ ]:
for a in range(2, qulQubitsClif):
    tt_tot = 0.0
    for t in range(times):
        t_start = time.perf_counter()

        state = QuantumState(a)
        state.set_zero_state()

        circuit = QC(a)

        for i in range(depth):
            amount = np.random.randint(0, 3)
            for j in range(amount):
                rGate = np.random.choice(clifford)
                i0 = np.random.randint(0, a)
                i1 = np.random.randint(0, a)
                while i0 == i1: i1 = np.random.randint(0, a)

                if rGate == 'H':
                    circuit.add_H_gate(i0)
                elif rGate == 'S':
                    circuit.add_S_gate(i0)
                elif rGate == 'X':
                    circuit.add_X_gate(i0)
                elif rGate == 'Y':
                    circuit.add_Y_gate(i0)
                elif rGate == 'Z':
                    circuit.add_Z_gate(i0)
                elif rGate == 'CX':
                    circuit.add_CNOT_gate(i0, i1)

        circuit.update_quantum_state(state)
        output = state
        del circuit
        del state

        t_stop = time.perf_counter()

        t_tot = t_stop - t_start
        tt_tot += t_tot

    t_avg = tt_tot / times
    qulClifT = np.append(qulClifT, [t_avg])

print(qulClifT)

This is so you can play with the amount of qubits and still can plot the graph without problems. 

In [ ]:
y4 = []
if qisQubitsClif < qulQubitsClif: 
    for i in range(qulQubitsClif - qisQubitsClif): 
        qisClifT = np.append(qisClifT, [np.nan])
    y4 = range(2, qulQubitsClif)
elif qisQubitsClif > qulQubitsClif:
    for i in range(qisQubitsClif - qulQubitsClif): 
        qulClifT = np.append(qulClifT, [np.nan])
    y4 = range(2, qisQubitsClif)
else:
    y4 = range(2, qisQubitsClif)

Below we will put these results in a graph in which you can see the time it takes to execute the random circuits on the increasing amount of qubits. 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=150) 

# the whole graph
ax.plot(y4, qisClifT, 'c', label='Qiskit', linewidth=2, marker='^', markersize=4)
ax.plot(y4, qulClifT, 'm', label='Qulacs', linewidth=2, marker='x', markersize=4)
plt.xticks(np.arange(2, qisQubitsClif, step=4))
plt.title('Time Clifford gates', fontsize=14)

plt.xlabel('$n$ qubits', fontsize=14)
plt.ylabel('time in seconds', fontsize=14)
ax.legend()

# zoomed in part
axins = inset_axes(ax, width="40%", height=1., loc='center right', borderpad=2)
mark_inset(ax, axins, loc1=3, loc2=4, fc='none', ec='0.5')

axins.plot(y4, qisClifT, 'c', label='Qiskit', linewidth=2, marker='^', markersize=4)
axins.plot(y4, qulClifT, 'm', label='Qulacs', linewidth=2, marker='x', markersize=4)

axins.set_xlim(2, 21)
axins.set_ylim(-0.01, 0.1)
plt.xticks(np.arange(2, 22, step=2))

# show the whole graph
plt.show()

In the benchmark for this implementation of random circuits you can see Qiskit can efficiently simulate more then 67 qubits. This means that we now know Qiskit is using the Gottesman-Knill theorem, while Qulacs is not. But Qiskit is not efficient with a low number of qubits, in that case Qulacs is better. 

### 2.5. Random circuit implementation 2 (Clifford gates): memory <a id='RCclifmem'></a>

In this part we will look at the memory it takes to execute the random circuit implementation with Clifford gates. We will look agian at the maximum resident set size. 

In [ ]:
qisClifM = np.array([], dtype=float)
qulClifM = np.array([], dtype=float)

The code for this random circuit implementation in Qiskit:

In [ ]:
for a in range(2, qisQubitsClif):
    rss_tot = 0.0
    for t in range(times):
        tracemalloc.start()
        q = QuantumRegister(a, 'q')
        c = ClassicalRegister(a, 'c')
        circ = QuantumCircuit(q, c)

        for i in range(depth):
            amount = np.random.randint(0, 3)

            for i in range(amount):
                # chosing a random Clifford gate
                rGate = np.random.choice(clifford)
                i0 = np.random.randint(0, a)
                i1 = np.random.randint(0, a)
                while i0 == i1: i1 = np.random.randint(0, a)

                if rGate == 'H':
                    circ.h(q[i0])
                elif rGate == 'S':
                    circ.s(q[i0])
                elif rGate == 'X':
                    circ.x(q[i0])
                elif rGate == 'Y':
                    circ.y(q[i0])
                elif rGate == 'Z':
                    circ.z(q[i0])
                elif rGate == 'CX':
                    circ.cx(q[i0], q[i1])

        circ.measure(q, c)

        backend = Aer.get_backend('qasm_simulator')
        job_sim = execute(circ, backend)
        sim_result = job_sim.result()

        snapshot = tracemalloc.take_snapshot()
        top_stats = snapshot.statistics('lineno')
        tracemalloc.stop()
        max_rss = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

        rss_tot += max_rss

    rss_avg = rss_tot / times

    qisClifM = np.append(qisClifM, [rss_avg])

print(qisClifM)

The code for this random circuit implementation in Qulacs:

In [ ]:
for a in range(2, qulQubitsClif):
    rss_tot = 0.0
    for t in range(times):
        tracemalloc.start()

        state = QuantumState(a)
        state.set_zero_state()

        circuit = QC(a)

        for i in range(depth):
            amount = np.random.randint(0, 3)
            for j in range(amount):
                rGate = np.random.choice(clifford)
                i0 = np.random.randint(0, a)
                i1 = np.random.randint(0, a)
                while i0 == i1: i1 = np.random.randint(0, a)

                if rGate == 'H':
                    circuit.add_H_gate(i0)
                elif rGate == 'S':
                    circuit.add_S_gate(i0)
                elif rGate == 'X':
                    circuit.add_X_gate(i0)
                elif rGate == 'Y':
                    circuit.add_Y_gate(i0)
                elif rGate == 'Z':
                    circuit.add_Z_gate(i0)
                elif rGate == 'CX':
                    circuit.add_CNOT_gate(i0, i1)

        circuit.update_quantum_state(state)
        output = state
        del circuit
        del state

        snapshot = tracemalloc.take_snapshot()
        top_stats = snapshot.statistics('lineno')
        tracemalloc.stop()
        max_rss = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

        rss_tot += max_rss

    rss_avg = rss_tot / times
    qulClifM = np.append(qulClifM, [rss_avg])

print(qulClifM)

This is so you can play with the amount of qubits and still can plot the graph without problems.

In [ ]:
y5 = []
if qisQubitsClif < qulQubitsClif: 
    for i in range(qulQubitsClif - qisQubitsClif): 
        qisClifM = np.append(qisClifM, [np.nan])
    y5 = range(2, qulQubitsClif)
elif qisQubitsClif > qulQubitsClif:
    for i in range(qisQubitsClif - qulQubitsClif): 
        qulClifM = np.append(qulClifM, [np.nan])
    y5 = range(2, qisQubitsClif)
else:
    y5 = range(2, qisQubitsClif)

Below we will put these results in a graph in which you can see the maximum resident set size it takes to execute the random circuits on the increasing amount of qubits. 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=150) 

# the whole graph
ax.plot(y5, qisClifM, 'c', label='Qiskit', linewidth=2, marker='^', markersize=4)
ax.plot(y5, qulClifM, 'm', label='Qulacs', linewidth=2, marker='x', markersize=4)
plt.xticks(np.arange(2, qisQubitsClif, step=4))
plt.title('Memory Clifford gates', fontsize=14)

plt.xlabel('$n$ qubits', fontsize=14)
plt.ylabel('maximum resident set size', fontsize=14)
ax.legend()

# show the whole graph
plt.show()

As you can see the amount of rss it is the same in qulacs and qiskit for the random clifford gates untill the 27th qubit then Qulacs uses a lot of memory, because computating on a large vector takes a lot of comute power. Qiskit has optimized these gates so this doesn't happen in this library. Qulacs is a libary that gives an out of memory error pretty fast you can also see this if you play around with the amount of qubits in this notebook.

### 2.6. Running on a QPU <a id='ibmQPU'></a>

Finally we will benchmark the random Clifford gates on a QPU to see how long the run time is. To obtain the token you need an IBMQ account https://quantum-computing.ibm.com/. After registering click on "My Account" there you will find the token to start running circuits on QPU's. For more information on how to run on a QPU with Qiskit: https://github.com/Qiskit/qiskit-ibmq-provider.

In [ ]:
token = '305f9ad0bd1baaacbaad000767927490e3924c38126a79ca6ca4ea96114d1e25cdf84d664cd15bb1798b793607fe852abdb675f1ec3cad3d2e1219d470295735' # put you token here
provider = IBMQ.enable_account(token)

We will run this circuit on the ibmq_london, because this queue is most of the time not as busy as the other QPU's. On the ibmq_london is a QPU with 5 qubits. If you want to use more then 5 qubits the ibmq_16_melbourne is recommend, this QPU had 16 qubits but is always busy. 

In [ ]:
qubitsQPU = 6
qpuRunT = np.array([], dtype=float)

Below we will run the random Clifford gates circuits on the QPU ibmq_london. This might take a while, because there is a queue. The time we measure is not the total time, but the time the circuit has run in the QPU.

In [ ]:
for k in range(2, qubitsQPU):
    q = QuantumRegister(k, 'q')
    c = ClassicalRegister(k, 'c')
    circ = QuantumCircuit(q, c)

    for i in range(depth):
        amount = np.random.randint(0, 3)

    for i in range(amount):
        # chosing a random Clifford gate
        rGate = np.random.choice(clifford)
        i0 = np.random.randint(0, k)
        i1 = np.random.randint(0, k)
        while i0 == i1: i1 = np.random.randint(0, k)

        if rGate == 'H':
            circ.h(q[i0])
        elif rGate == 'S':
            circ.s(q[i0])
        elif rGate == 'X':
            circ.x(q[i0])
        elif rGate == 'Y':
            circ.y(q[i0])
        elif rGate == 'Z':
            circ.z(q[i0])
        elif rGate == 'CX':
            circ.cx(q[i0], q[i1])

    circ.measure(q, c)

    backend = provider.get_backend('ibmq_london') # to run on the ibmq_london device
    job = execute(circ, backend=backend, shots=1024)
    qpuRunT = np.append(qpuRunT, [job.result().time_taken])
    print(f'{k}: {job.result().time_taken}')

print(qpuRunT)

Below we will plot the results of the run time with the increasing amount of qubits.

In [ ]:
y6 = range(2, qubitsQPU)
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=150) 

# the whole graph
ax.plot(y6, qpuRunT, 'c', label='Qiskit', linewidth=2, marker='^', markersize=4)
plt.xticks(np.arange(2, qubitsQPU, step=1))
plt.title('Clifford gates on the ibmq_london', fontsize=14)

plt.xlabel('$n$ qubits', fontsize=14)
plt.ylabel('time in seconds', fontsize=14)
ax.legend()

# show the whole graph
plt.show()

As you can see the run time is a lot slower then when we simulated the random Clifford gates in section 2.3.. 

## 3. Conclusion <a id='conclusion'></a>

To further develop quantum algorithms it is necessary to have quantum software. Already there are a lot of different quantum software and libraries, therefore it would be great to look at the performances of some libraries. In this notebook we saw that a lot of research papers use random circuits to benchmark quantum and with quantum software and libraries most compare the time and memory it takes to execute a circuit. The problem with quantum simulators is that there are a lot and they are all different, this means it is difficult to compare these software and libraries with each other in a benchmark. 

In this notebook we have looked at the two python libraries Qiskit and Qulacs. These libraries have been chosen, because they are different from each other and interesting for researchers. Qiskit is a full-stack library with a lot of documentation and beginners guides, Qulacs is said to run fast and can run on a GPU. 

From the benchmarks in this notebook the conclusion is that Qulacs for circuits with 20 or less qubits the fastest is, because it compresses circuits to simulate them faster. But Qiskit is best for using Clifford gates on more then 20 qubits. Qulacs uses more maximum resident set size then Qiskit. 

## References <a id='references'></a>

1. Aaronson, S., & Gottesman, D. (2004). Improved simulation of stabilizer circuits.Physical ReviewA,70(5), 1–15. https://doi.org/10.1103/physreva.70.052328

2. Abraham,  H.,  Akhalwaya,  I.  Y.,  Aleksandrowicz,  G.,  Alexander,  T.,  Alexandrowics,  G.,  Arbel,E.,  Asfaw,  A.,  Azaustre,  C.,  AzizNgoueya,  Barkoutsos,  P.,  Barron,  G.,  Bello,  L.,  Ben-Haim,  Y.,  Bevenius,  D.,  Bishop,  L.  S.,  Bosch,  S.,  Bravyi,  S.,  Bucher,  D.,  Cabrera,  F.,. . .  yotamvakninibm. (2019). Qiskit: An open-source framework for quantum computing.https://doi.org/10.5281/zenodo.2562110

3. Asfaw, A., Bello, L., Ben-Haim, Y., Bravyi, S., Capelluto, L., Vazquez, A. C., Ceroni, J., Chen, R.,Frisch, A., Gambetta, J., Garion, S., Gil, L., Gonzalez, S. D. L. P., Harkins, F., Imamichi,T., McKay, D., Mezzacapo, A., Minev, Z., Movassagh, R., . . .  Wootton, J. (2020). Learnquantum computation using qiskit. http://community.qiskit.org/textbook

4. Emerson, J. (2004). Pseudo-random unitary operators for quantum information processing.Sci-ence,302(5653). https://doi.org/10.1126/science.1090790

5. Fingerhuth,  M.,  Babej,  T.,  &  Wittek,  P.  (2018).  Open  source  software  in  quantum  computing.PLOS ONE,13(12), 1–28. https://doi.org/10.1371/journal.pone.0208561

6. Jones, T., Brown, A., Bush, I., & Benjamin, S. (2018). Quest and high performance simulation ofquantum computers. https://arxiv.org/pdf/1802.08032.pdf

7. Qulacs. (2018). https://github.com/qulacs/qulacs